<a href="https://colab.research.google.com/github/meishihna/GITHUB-COLAB/blob/main/%E6%89%BE%E5%87%BA%E9%95%B7%E7%B4%85K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**長紅K強勢股**

```
上傳symbol_pool.csv



In [ ]:
!pip install lineTool
!pip install twstock

In [2]:
import pandas as pd
import requests
import lineTool
import twstock
import time
import csv
import os

In [ ]:
folder_name = "stock_data"
# 检查文件夹是否存在，如果不存在则创建
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

date_range = [i.strftime('%Y%m%d') for i in pd.date_range('2024-01-01','2024-03-31')]
# 因為資料是從最舊的開始抓取，所以將list倒轉
date_range.reverse()

for d in date_range:
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'
    formdata = {
        'response': 'csv',
        'date': d,
        'type': 'ALLBUT0999',
    }

    # 取得資料並且解析
    r = requests.get(url, params=formdata)
    r.text.encode('utf8')
    cr = csv.reader(r.text.splitlines(), delimiter=',')
    my_list = list(cr)

    # 開始做資料整理
    if len(my_list) > 0:
        for i in range(len(my_list)):
            if len(my_list[i]) > 0:
                if my_list[i][0] == '證券代號':
                    new_list = my_list[i:]
                    break
        for j in range(len(new_list)):
            if j != 0:
                try:
                    new_list[j][0] = new_list[j][0].split('"')[1]
                except:
                    break
        df = pd.DataFrame(new_list[1:], columns=new_list[0])
        # TODO 這邊請手動新增資料夾，並將檔案存在指定資料夾內，否則檔案會超多份哦
        filename = 'stock_data/price_{}.csv'.format(d)
        # 寫入csv
        df.to_csv(filename)
        print('Date {} file done'.format(d))
        time.sleep(3.2)
    else:
        print('Date {} no data'.format(d))

### 從產業分類挑選候選股票清單
# symbol_pool = pd.read_csv("symbol_pool.csv", dtype=str)
# symbol_list = list(symbol_pool.loc[symbol_pool["industry"].isin(["金融保險業"]), "symbolId"])

### 全選
symbol_pool = pd.read_csv("symbol_pool.csv", dtype=str)
symbol_list = symbol_pool["symbolId"]
## 指定選股資料起訖日期
date_from = "2019-01-01"
date_to = "2024-12-31"
## 生成交易日曆
date_range = pd.date_range(date_from, date_to)
date_range_str = [date.strftime("%Y%m%d") for date in date_range]
dir_and_files = os.listdir("stock_data")
all_open_dates = [
    f.split("_")[-1].split(".")[0]
    for f in dir_and_files
    if f[:5] == "price" and f[-4:] == ".csv"
]

open_dates = [date for date in date_range_str if date in all_open_dates]

## 備用資料：讀取並整理資料格式
path_price = {date: "stock_data/price_{}.csv".format(date) for date in open_dates}
price_df = pd.DataFrame()
for date in path_price.keys():
    one_day_price = pd.read_csv(path_price[date])
    one_day_price.index = [date] * len(one_day_price)
    price_df = price_df.append(one_day_price.loc[one_day_price["證券代號"].isin(symbol_list)])
price_df.index = pd.to_datetime(price_df.index)

## 指定邏輯參數
vol_ma_p = 20
big_vol_multiplier = 2
k_body_floor = 0.03
k_body_ma_p = 20
k_body_multiplier = 2
vol_floor = 100

price_df["開盤價"] = pd.to_numeric(price_df["開盤價"].str.replace("--","").str.replace(",",""))
price_df["收盤價"] = pd.to_numeric(price_df["收盤價"].str.replace("--","").str.replace(",",""))
price_df["最高價"] = pd.to_numeric(price_df["最高價"].str.replace("--","").str.replace(",",""))
price_df["最低價"] = pd.to_numeric(price_df["最低價"].str.replace("--","").str.replace(",",""))
price_df["本益比"] = pd.to_numeric(price_df["本益比"].str.replace("--","").str.replace(",",""))
price_df["成交股數"] = pd.to_numeric(price_df["成交股數"].str.replace("--","").str.replace(",",""))
price_df["證券代號"] = pd.to_numeric(price_df["證券代號"])

### 從 price_df 萃取出開盤價 DataFrame
open_df = price_df.pivot_table(columns = "證券代號", index = price_df.index , values = '開盤價' )
### 從 price_df 萃取出收盤價 DataFrame
close_df = price_df.pivot_table(index=price_df.index, columns="證券代號", values="收盤價")
### 從 price_df 萃取出成交量 DataFrame
vol_df = price_df.pivot_table(index=price_df.index, columns="證券代號", values="成交股數")

### 挑出爆量的股票
vol_ma_df = vol_df.rolling(vol_ma_p).mean()
big_vol_filter = pd.DataFrame(
    np.where(vol_df > vol_ma_df * big_vol_multiplier, True, False),
    index=vol_df.index,
    columns=vol_df.columns,
)

### 挑出長紅的股票
k_body_range = (close_df - open_df) / open_df  # 正的是紅K、負的是黑K
k_body_range_ma = k_body_range.rolling(k_body_ma_p).mean()

big_rise_k_filter = pd.DataFrame(
    np.where(
        (k_body_range > k_body_floor)
        & (k_body_range > k_body_range_ma * k_body_multiplier),
        True,
        False,
    ),
    index=k_body_range.index,
    columns=k_body_range.columns,
)

### 交易量濾網：捨棄月均量太小的股票
avg_vol_df = vol_df.rolling(20).mean()
vol_share_floor = vol_floor * 1000
vol_filter = pd.DataFrame(np.where(avg_vol_df > vol_share_floor, True, False),
    index=avg_vol_df.index,
    columns=avg_vol_df.columns,)

### 取交集，得到最終結果
final_select = big_vol_filter & big_rise_k_filter & vol_filter
latest_result = final_select.loc[:, final_select.iloc[-1]].columns

def get_two_float(f_str, n):
    a, b, c = f_str.partition('.')
    c = c[:n]
    return ".".join([a, c])

url = "https://notify-api.line.me/api/notify"
headers = {'Authorization': 'Bearer ' + '0iFwGtNsQXgT0BPCFyI7vqUw1bxGiLsUhl79hXcXVmm'}

for code in latest_result:
    # 将整数转换为字符串，并添加到stock_codes列表中
    stock_codes.append(str(code))

messages = {}
messages = []
for code in latest_result:
        try:
            stock = twstock.realtime.get(str(code))
            if 'info' in stock and 'name' in stock['info'] :
                name = stock['info']['name']
                data = (f'\n{code}  【{name}】')
                messages.append(data)
        except Exception as e:
            print(f"Error occurred: {e}")
            continue
    # 合并所有消息
combined_message = ''.join(messages)
payload = {'message': combined_message}
response = requests.post(url, headers=headers, data=payload)
print(response.text)

Date 20240304 file done
Date 20240303 no data
Date 20240302 no data
Date 20240301 file done
Date 20240229 file done
Date 20240228 no data
Date 20240227 file done
Date 20240226 file done
Date 20240225 no data
Date 20240224 no data
Date 20240223 file done
Date 20240222 file done
Date 20240221 file done
Date 20240220 file done
Date 20240219 file done
Date 20240218 no data
Date 20240217 no data
Date 20240216 file done
Date 20240215 file done
Date 20240214 no data
Date 20240213 no data
Date 20240212 no data
Date 20240211 no data
Date 20240210 no data
Date 20240209 no data
Date 20240208 no data
Date 20240207 no data
Date 20240206 no data
Date 20240205 file done
Date 20240204 no data
Date 20240203 no data
Date 20240202 file done
Date 20240201 file done
Date 20240131 file done
Date 20240130 file done
Date 20240129 file done
Date 20240128 no data
Date 20240127 no data
Date 20240126 file done
Date 20240125 file done
Date 20240124 file done
Date 20240123 file done
Date 20240122 file done
Date 202

####練習

In [ ]:
pd.read_csv("symbol_pool.csv")